## For TPC-DS dataset with the following query tempalte:
SELECT ss_store_sk, AF(ss_sales_price)

FROM store_sales

WHERE ss_sold_date_sk BETWEEN low AND high

GROUP BY ss_store_sk

In [1]:
from dbestclient.executor.executor import SqlExecutor

## Parameter Initialization

In [2]:
sqlExecutor = SqlExecutor()
sqlExecutor.execute("set csv_split_char='|'")
sqlExecutor.execute("set encoder='embedding'")
sqlExecutor.execute("set n_epoch=20")
sqlExecutor.execute(
    "set table_header="
    + "'ss_sold_date_sk|ss_sold_time_sk|ss_item_sk|ss_customer_sk|ss_cdemo_sk|ss_hdemo_sk|"
    + "ss_addr_sk|ss_store_sk|ss_promo_sk|ss_ticket_number|ss_quantity|ss_wholesale_cost|"
    + "ss_list_price|ss_sales_price|ss_ext_discount_amt|ss_ext_sales_price|"
    + "ss_ext_wholesale_cost|ss_ext_list_price|ss_ext_tax|ss_coupon_amt|ss_net_paid|"
    + "ss_net_paid_inc_tax|ss_net_profit|none'"
)

Local mode is on, as no slaves are provided.
start loading pre-existing models.
Loaded 6 models. time cost  0.299734 s
OK, csv_split_char is updated.
OK, encoder is updated.
OK, n_epoch is updated.
OK, table_header is updated.


## Model Creation

In [3]:
# sqlExecutor.execute("drop table template1")  # drop the model
sqlExecutor.execute(
    "create table template1(ss_sales_price real, ss_sold_date_sk real) from '/home/quincy/Documents/workspace/data/tpcds/1t/ss_1m.csv' GROUP BY ss_store_sk method uniform size 0.0003472"
)

Start creating model template1
The given table is treated as a uniform sample, and it is obtained with sampling rate 0.0003472
Reading data file...
get frequency info from data....
Starting training kde models for model template1
training regression...
********************************************************************************
finish training embedding.
embedding inference...
start normalizing data...
transform data from MDN training...
finish transforming data from MDN training...
< Epoch 0
< Epoch 1
< Epoch 2
< Epoch 3
< Epoch 4
< Epoch 5
< Epoch 6
< Epoch 7
< Epoch 8
< Epoch 9
< Epoch 10
< Epoch 11
< Epoch 12
< Epoch 13
< Epoch 14
< Epoch 15
< Epoch 16
< Epoch 17
< Epoch 18
< Epoch 19
Finish regression training.
training density...
********************************************************************************
finish training embedding.
< Epoch 0
< Epoch 1
< Epoch 2
< Epoch 3
< Epoch 4
< Epoch 5
< Epoch 6
< Epoch 7
< Epoch 8
< Epoch 9
< Epoch 10
< Epoch 11
< Epoch 12
< Epoch 1

## Query Serving

In [3]:
predictions = sqlExecutor.execute(
    "select ss_store_sk, sum(ss_sales_price)  from template1 where   2451119  <=ss_sold_date_sk<= 2451483  group by ss_store_sk"
)

# sqlExecutor.execute("drop table template1")

OK
   0            1
     2.668114e+08
   1 4.316469e+07
  10 4.301621e+07
 100 4.437739e+07
1000 4.324589e+07
 103 4.226702e+07
 104 4.498155e+07
 106 4.271063e+07
 109 4.340035e+07
 110 4.409871e+07
 112 4.369547e+07
 115 4.485836e+07
 116 4.329969e+07
 118 4.354693e+07
 121 4.349904e+07
 122 4.505181e+07
 124 4.364896e+07
 127 4.496908e+07
 128 4.555428e+07
  13 4.262637e+07
 130 4.356832e+07
 133 4.379093e+07
 134 4.474557e+07
 136 4.280454e+07
 139 4.301845e+07
  14 4.485438e+07
 140 4.387837e+07
 142 4.514344e+07
 145 4.486742e+07
 146 4.593422e+07
 148 4.485955e+07
 151 4.244370e+07
 152 4.395475e+07
 154 4.374643e+07
 157 4.363563e+07
 158 4.440847e+07
  16 4.477698e+07
 160 4.559933e+07
 163 4.373289e+07
 164 4.100966e+07
 166 4.399090e+07
 169 4.225455e+07
 170 4.478090e+07
 172 4.456956e+07
 175 4.441835e+07
 176 4.372149e+07
 178 4.506843e+07
 181 4.202162e+07
 182 4.331495e+07
 184 4.448747e+07
 187 4.607270e+07
 188 4.303476e+07
  19 4.432108e+07
 190 4.208626e+07
 193 4.

## HIVE query
select ss_store_sk, count(ss_sales_price),sum(ss_sales_price),avg(ss_sales_price)  from store_sales_1t where  ss_sold_date_sk BETWEEN 2451119  AND 2451483  group by ss_store_sk